<a href="https://colab.research.google.com/github/kkc0122/Machine-Learning/blob/main/1_DART_API_%ED%99%9C%EC%9A%A9_%EB%B6%80%EC%8B%A4%EA%B8%B0%EC%97%85_%EC%9E%AC%EB%AC%B4%EC%A0%9C%ED%91%9C_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%B6%94%EC%B6%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
!pip install dart_fss
import dart_fss as dart
import requests
from bs4 import BeautifulSoup
import concurrent.futures
import time

# DART API 키 설정
api_key = 'b466c6a530c6f3ff91f41d5250ed68ddbdf45d1f'
dart.set_api_key(api_key=api_key)

# 기업 리스트 로드
corp_list = dart.get_corp_list()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.7/145.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.1/201.1 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 38.6 MB/s eta 0:00:00
  Attempting uninstall: termcolor
    Found existing installation: termcolor 2.5.0
    Uninstalling termcolor-2.5.0:
      Successfully uninstalled termcolor-2.5.0


In [ ]:
# 캐싱 및 데이터 추출 함수
def fetch_or_load_fs(corp_name='스마트솔루션즈', year=2023):
    filename = f'{corp_name}_{year}.pkl'

    if os.path.exists(filename):
        print(f'{filename} 로드 중...')
        return pd.read_pickle(filename)
    else:
        print(f'{filename} 다운로드 중...')
        corp = corp_list.find_by_corp_name(corp_name, exactly=True)
        if not corp:
            print(f'기업 {corp_name}를 찾을 수 없습니다.')
            return None

        corp = corp[0]
        try:
            fs = corp.extract_fs(bgn_de=f'{year}0101')
            fs.dataframe.to_pickle(filename)
            print(f'{filename} 저장 완료')
            return fs.dataframe
        except Exception as e:
            print(f'오류 발생: {corp_name} - {e}')
            return None

In [ ]:
# 병렬 처리 함수
from concurrent.futures import ProcessPoolExecutor

def parallel_fetch_fs(corp_names, year):
    with ProcessPoolExecutor(max_workers=5) as executor:
        futures = {executor.submit(fetch_or_load_fs, corp_name, year): corp_name for corp_name in corp_names}
        results = {}
        for future in concurrent.futures.as_completed(futures):
            corp_name = futures[future]
            try:
                result = future.result()
                results[corp_name] = result
            except Exception as e:
                print(f'오류 발생: {corp_name} - {e}')
        return results

In [ ]:
# 테스트 실행
if __name__ == "__main__":
    corp_names = ['바이오빌']  # 처리할 기업 리스트
    year = '2021'

    # 병렬 처리 실행
    start_time = time.time()
    results = parallel_fetch_fs(corp_names, year)
    end_time = time.time()

    print(f"총 실행 시간: {end_time - start_time:.2f}초")

    # 결과 확인
    for corp_name, df in results.items():
        if df is not None:
            print(f"{corp_name} 데이터 로드 완료")

바이오빌_2021.pkl 다운로드 중...


Annual reports:   0%|          | 0/2 [00:00<?, ?report/s]

오류 발생: 바이오빌 - 'FinancialStatement' object has no attribute 'dataframe'
총 실행 시간: 29.35초
